In [1]:
import pymongo
from dotenv import load_dotenv
import os
import time

load_dotenv()
mongodb_uri = os.getenv("MONGODB_URI")
client = pymongo.MongoClient(mongodb_uri)
db = os.environ.get("MONGODB_DB")
locations = db["locations"]

In [2]:
import itertools
coords_list = [ x['coordinates'] for x in list(locations.find({}, {"_id": 0, "coordinates": 1}))]
coords_combos = list(itertools.combinations(coords_list, 2))
print(len(coords_combos))

5995


In [3]:
haversine_distances = []
import haversine as hs

start = time.time()

for ele in coords_combos*1000:
    haversine_distances.append(hs.haversine(ele[0], ele[1], unit=hs.Unit.METERS))

haversine_time = time.time() - start
print(haversine_time)

12.671452522277832


In [4]:
pc_distances = []
from math import pi, sqrt, cos

start = time.time()

for ele in coords_combos*1000:
    lat1 = ele[0][0] * (pi/180)
    lon1 = ele[0][1] * (pi/180)
    lat2 = ele[1][0] * (pi/180)
    lon2 = ele[1][1] * (pi/180)

    a = (pi/2) - lat1  
    b = (pi/2) - lat2  
    c = sqrt( (a**2) + (b**2) - (2 * a * b * cos(lon2 - lon1) ))   
    pc_distances.append(6_371_000  * c)

pc_time = time.time() - start
print(pc_time)

9.354665994644165


In [5]:
pyth_distances = []
from math import pi, sqrt, cos

start = time.time()


for ele in coords_combos*1000:
    lat1 = ele[0][0]  
    lon1 = ele[0][1]  
    lat2 = ele[1][0] 
    lon2 = ele[1][1] 

    avgLatDeg = (lat1 + lat2) / 2
    avgLat = avgLatDeg * (pi/180)

    d_ew = (lon2 - lon1) * cos(avgLat)
    d_ns = (lat2 - lat1)
    approxDegreesSq = (d_ew * d_ew) + (d_ns * d_ns)
    d_degrees = sqrt(approxDegreesSq)
    EarthAvgMeterPerGreatCircleDegree = (6371000 * 2 * pi) / 360
    pyth_distances.append(d_degrees * EarthAvgMeterPerGreatCircleDegree )


pyth_time = time.time() - start
print(pyth_time)

In [ ]:
import numpy as np

def mape(y_test, pred):
    y_test, pred = np.array(y_test), np.array(pred)
    mape = np.mean(np.abs((y_test - pred) / y_test))
    return mape*100

In [ ]:
print(mape(haversine_distances, pc_distances))
print(mape(haversine_distances, pyth_distances))

4.858501884243121
0.001829012508716104
